1. pip install python-twitter
1. test images: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html
1. ImageNet Labels: https://gist.github.com/yrevar/942d3a0ac09ec9e5eb3a

## Imports and Intro

In [ ]:
import twitter
import torch
import torchvision
import numpy as np

class Object(object): pass

## Load Model

In [ ]:
t = Object()

In [ ]:
t.model = torchvision.models.alexnet(pretrained=True)

In [ ]:
# !ls ../input/ants-bees/train/ants/_ants.jpg

## Convert Model

Convert the model to TorchScript:
1. Load an image
2. Run jit.trace

In [ ]:
torch.Tensor([1, 2, 3])

In [ ]:
t.img_t = torch.rand(1, 3, 224, 224)

In [ ]:
t.model.eval()
t.nn_script = torch.jit.trace(t.model, t.img_t)

In [ ]:
import ml2rt
t.nn_script.eval()
ml2rt.save_torch(t.nn_script, "model.pt")
t.loaded_nn = ml2rt.load_model("model.pt")

Load model into RedisAI

In [ ]:
import redisai

In [ ]:
# !docker run -p 6379:6379 -it --rm redisai/redisai

In [ ]:
t.redis = redisai.Client()
try:
    t.redis.loadbackend('TORCH', 'redisai_torch/redisai_torch.so')
except:
    pass

In [ ]:
t.redis.modelset('resnet', 
                 redisai.Backend.torch, 
                 redisai.Device.cpu,
                 t.loaded_nn)

## Load Test Image

test images: https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [ ]:
import PIL
import ml2rt

t.img = PIL.Image\
    .open("../input/ants-bees/train/ants/_ants.jpg")\
    .convert("RGB")
t.transforms = torchvision.transforms.Compose([
    torchvision.transforms.Resize(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
])
t.img_t = t.transforms(t.img).unsqueeze(0)
t.img_t.shape

In [ ]:
t.img

## Check Predictions

In [ ]:
t.redis.tensorset('image', t.img_t.numpy())
t.redis.modelrun('resnet', 'image', 'pred')
t.pred = t.redis.tensorget('pred')[0]
t.pred

## Convert to probabilities

In [ ]:
def softmax(p):
    t = np.exp(p)
    s = t.sum()
    return t / s

In [ ]:
t.proba = softmax(t.pred)

In [ ]:
def get_top_5(preds):
    preds = preds.copy()
    result = np.zeros(5).astype(np.int32)
    for i in range(result.shape[0]):
        max_ind = preds.argmax()
        result[i] = int(max_ind)
        preds[max_ind] = -1
    return result

In [ ]:
t.top5 = get_top_5(t.proba)
t.top5

In [ ]:
import ast, requests 

t.labels = ast.literal_eval(
    requests.get('https://gist.githubusercontent.com/yrevar/942d3a0ac09ec9e5eb3a/raw/238f720ff059c1f82f368259d1ca4ffa5dd8f9f5/imagenet1000_clsidx_to_labels.txt').text
)

In [ ]:
for i in get_top_5(t.proba):
    print(t.labels[i], t.proba[i])

## Add Twitter Loop

https://developer.twitter.com/apps

In [ ]:
import os

t.api = twitter.Api(
    consumer_key=os.environ['TWITTER_CONSUMER_KEY'],
    consumer_secret=os.environ['TWITTER_CONSUMER_SECRET'],
    access_token_key=os.environ['TWITTER_TOKEN_KEY'],
    access_token_secret=os.environ['TWITTER_TOKEN_SECRET'])

In [ ]:
# print(t.api.VerifyCredentials())

In [ ]:
t.t = t.api.GetUserTimeline(screen_name="alexonsoftware", 
                        count=10, 
                        include_rts=False,
                        exclude_replies=True)

In [ ]:
since_id = t.t[0].id
since_id -= 1

Get new tweets:

In [ ]:
t.t = t.api.GetUserTimeline(screen_name="alexonsoftware", 
                        count=10, 
                        include_rts=False,
                        exclude_replies=True,
                        since_id=since_id)
t.t

## Run the loop

In [ ]:
import io, urllib.request, time

msg = Object()
msg.running = True

while msg.running:
    msg.t = t.api.GetUserTimeline(screen_name="alexonsoftware", 
                        count=10, 
                        include_rts=False,
                        exclude_replies=True,
                        since_id=since_id)
    if len(msg.t) > 0:
        for msg.tweet in msg.t:
            if 'media' in msg.tweet.AsDict():
                msg.media_list = msg.tweet.AsDict()['media']
                for msg.pic in msg.media_list:
                    msg.img = PIL.Image.open(
                        io.BytesIO(
                            urllib.request.urlopen(
                                msg.pic['media_url']).read())).convert('RGB')
                    msg.img_t = t.transforms(msg.img).unsqueeze(0)
                    t.redis.tensorset('image', msg.img_t.numpy())
                    t.redis.modelrun('resnet', 'image', 'pred')
                    
                    msg.pred = t.redis.tensorget('pred')[0]
                    
                    msg.pred = softmax(msg.pred)

                    msg.ind = get_top_5(msg.pred)
                    
                    result = ''
                    for msg.p in msg.ind:
                        result += f"{t.labels[msg.p]} : {msg.pred[msg.p]}\n"
                    print(result)
                    t.api.PostUpdate(
                        status=result,
                        in_reply_to_status_id=msg.tweet.id
                    )
            since_id = max(since_id, msg.tweet.id)
    time.sleep(1)
    msg.running = True